## Importing the Packages

In [2]:
import numpy as np

In [3]:
np.set_printoptions(suppress = True, linewidth = 100, precision = 2)

## Importing the Data

In [5]:
raw_data_np = np.genfromtxt("loan-data.csv", delimiter = ';', skip_header = 1, autostrip = True, encoding='latin1')
raw_data_np

array([[48010226.  ,         nan,    35000.  , ...,         nan,         nan,     9452.96],
       [57693261.  ,         nan,    30000.  , ...,         nan,         nan,     4679.7 ],
       [59432726.  ,         nan,    15000.  , ...,         nan,         nan,     1969.83],
       ...,
       [50415990.  ,         nan,    10000.  , ...,         nan,         nan,     2185.64],
       [46154151.  ,         nan,         nan, ...,         nan,         nan,     3199.4 ],
       [66055249.  ,         nan,    10000.  , ...,         nan,         nan,      301.9 ]])

## Checking for Incomplete Data

In [7]:
np.isnan(raw_data_np).sum()

88005

In [8]:
temporary_fill = np.nanmax(raw_data_np) + 1
temporary_mean = np.nanmean(raw_data_np, axis = 0)

/var/folders/4w/l345c62j7n7_bv_1yz1bhqc80000gn/T/ipykernel_12171/3983241459.py:2: RuntimeWarning: Mean of empty slice
  temporary_mean = np.nanmean(raw_data_np, axis = 0)


In [9]:
temporary_mean

array([54015809.19,         nan,    15273.46,         nan,    15311.04,         nan,       16.62,
            440.92,         nan,         nan,         nan,         nan,         nan,     3143.85])

In [10]:
temporary_stats = np.array([np.nanmin(raw_data_np, axis = 0),
                           temporary_mean,
                           np.nanmax(raw_data_np, axis = 0)])

/var/folders/4w/l345c62j7n7_bv_1yz1bhqc80000gn/T/ipykernel_12171/2183409543.py:1: RuntimeWarning: All-NaN slice encountered
  temporary_stats = np.array([np.nanmin(raw_data_np, axis = 0),
/var/folders/4w/l345c62j7n7_bv_1yz1bhqc80000gn/T/ipykernel_12171/2183409543.py:3: RuntimeWarning: All-NaN slice encountered
  np.nanmax(raw_data_np, axis = 0)])


In [11]:
temporary_stats

array([[  373332.  ,         nan,     1000.  ,         nan,     1000.  ,         nan,        6.  ,
              31.42,         nan,         nan,         nan,         nan,         nan,        0.  ],
       [54015809.19,         nan,    15273.46,         nan,    15311.04,         nan,       16.62,
             440.92,         nan,         nan,         nan,         nan,         nan,     3143.85],
       [68616519.  ,         nan,    35000.  ,         nan,    35000.  ,         nan,       28.99,
            1372.97,         nan,         nan,         nan,         nan,         nan,    41913.62]])

## Splitting the Dataset

### Splitting the Columns

In [14]:
columns_strings = np.argwhere(np.isnan(temporary_mean)).squeeze()
columns_strings

array([ 1,  3,  5,  8,  9, 10, 11, 12])

In [15]:
columns_numeric = np.argwhere(np.isnan(temporary_mean) == False).squeeze()
columns_numeric

array([ 0,  2,  4,  6,  7, 13])

### Re-importing the Dataset

In [17]:
loan_data_strings = np.genfromtxt("loan-data.csv",
                                  delimiter = ';',
                                  skip_header = 1,
                                  autostrip = True, 
                                  usecols = columns_strings,
                                  dtype = str)
loan_data_strings

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xfe in position 341: invalid start byte

In [62]:
loan_data_numeric = np.genfromtxt("loan-data.csv",
                                  delimiter = ';',
                                  autostrip = True,
                                  skip_header = 1,
                                  usecols = columns_numeric,
                                  filling_values = temporary_fill)
loan_data_numeric

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xfe in position 341: invalid start byte

### The Names of the Columns

In [ ]:
header_full = np.genfromtxt("loan-data.csv",
                            delimiter = ';',
                            autostrip = True,
                            skip_footer = raw_data_np.shape[0],
                            dtype = np.str)
header_full

In [ ]:
header_strings, header_numeric = header_full[columns_strings], header_full[columns_numeric]

In [ ]:
header_strings

In [ ]:
header_numeric

## Creating Checkpoints:

In [ ]:
def checkpoint(file_name, checkpoint_header, checkpoint_data):
    np.savez(file_name, header = checkpoint_header, data = checkpoint_data)
    checkpoint_variable = np.load(file_name + ".npz")
    return(checkpoint_variable)

In [ ]:
checkpoint_test = checkpoint("checkpoint-test", header_strings, loan_data_strings)

In [ ]:
checkpoint_test['data']

In [ ]:
np.array_equal(checkpoint_test['data'], loan_data_strings)

## Manipulating String Columns

In [ ]:
header_strings

In [ ]:
header_strings[0] = "issue_date"

In [ ]:
loan_data_strings

### Issue Date

In [ ]:
np.unique(loan_data_strings[:,0])

In [ ]:
loan_data_strings[:,0] = np.chararray.strip(loan_data_strings[:,0], "-15")

In [ ]:
np.unique(loan_data_strings[:,0])

In [ ]:
months = np.array(['', 'Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'])

In [ ]:
for i in range(13):
        loan_data_strings[:,0] = np.where(loan_data_strings[:,0] == months[i],
                                          i,
                                          loan_data_strings[:,0])

In [ ]:
np.unique(loan_data_strings[:,0])

### Loan Status

In [ ]:
header_strings

In [ ]:
np.unique(loan_data_strings[:,1])

In [ ]:
np.unique(loan_data_strings[:,1]).size

In [ ]:
status_bad = np.array(['','Charged Off','Default','Late (31-120 days)'])

In [ ]:
loan_data_strings[:,1] = np.where(np.isin(loan_data_strings[:,1], status_bad),0,1)

In [ ]:
np.unique(loan_data_strings[:,1])

### Term

In [ ]:
header_strings

In [ ]:
np.unique(loan_data_strings[:,2])

In [ ]:
loan_data_strings[:,2] = np.chararray.strip(loan_data_strings[:,2], " months")
loan_data_strings[:,2]

In [ ]:
header_strings[2] = "term_months"

In [ ]:
loan_data_strings[:,2] = np.where(loan_data_strings[:,2] == '', 
                                  '60', 
                                  loan_data_strings[:,2])
loan_data_strings[:,2]

In [ ]:
np.unique(loan_data_strings[:,2])

### Grade and Subgrade

In [ ]:
header_strings

In [ ]:
np.unique(loan_data_strings[:,3])

In [ ]:
np.unique(loan_data_strings[:,4])

#### Filling Sub Grade

In [ ]:
for i in np.unique(loan_data_strings[:,3])[1:]:
    loan_data_strings[:,4] = np.where((loan_data_strings[:,4] == '') & (loan_data_strings[:,3] == i),
                                      i + '5',
                                      loan_data_strings[:,4])

In [ ]:
np.unique(loan_data_strings[:,4], return_counts = True)

In [ ]:
loan_data_strings[:,4] = np.where(loan_data_strings[:,4] == '',
                                  'H1',
                                  loan_data_strings[:,4])

In [ ]:
np.unique(loan_data_strings[:,4])

#### Removing Grade

In [ ]:
loan_data_strings = np.delete(loan_data_strings, 3, axis = 1)

In [ ]:
loan_data_strings[:,3]

In [ ]:
header_strings = np.delete(header_strings, 3)

In [ ]:
header_strings[3]

#### Converting Sub Grade

In [ ]:
np.unique(loan_data_strings[:,3])

In [ ]:
keys = list(np.unique(loan_data_strings[:,3]))                         
values = list(range(1, np.unique(loan_data_strings[:,3]).shape[0] + 1)) 
dict_sub_grade = dict(zip(keys, values))

In [ ]:
dict_sub_grade

In [ ]:
for i in np.unique(loan_data_strings[:,3]):
        loan_data_strings[:,3] = np.where(loan_data_strings[:,3] == i, 
                                          dict_sub_grade[i],
                                          loan_data_strings[:,3])

In [ ]:
np.unique(loan_data_strings[:,3])

### Verification Status

In [ ]:
header_strings

In [ ]:
np.unique(loan_data_strings[:,4])

In [ ]:
loan_data_strings[:,4] = np.where((loan_data_strings[:,4] == '') | (loan_data_strings[:,4] == 'Not Verified'), 0, 1)

In [ ]:
np.unique(loan_data_strings[:,4])

### URL

In [ ]:
loan_data_strings[:,5]

In [ ]:
np.chararray.strip(loan_data_strings[:,5], "https://www.lendingclub.com/browse/loanDetail.action?loan_id=")

In [ ]:
loan_data_strings[:,5] = np.chararray.strip(loan_data_strings[:,5], "https://www.lendingclub.com/browse/loanDetail.action?loan_id=")

In [ ]:
header_full

In [ ]:
loan_data_numeric[:,0].astype(dtype = np.int32)

In [ ]:
loan_data_strings[:,5].astype(dtype = np.int32)

In [ ]:
np.array_equal(loan_data_numeric[:,0].astype(dtype = np.int32), loan_data_strings[:,5].astype(dtype = np.int32))

In [ ]:
loan_data_strings = np.delete(loan_data_strings, 5, axis = 1)
header_strings = np.delete(header_strings, 5)

In [ ]:
loan_data_strings[:,5]

In [ ]:
header_strings

In [ ]:
loan_data_numeric[:,0]

In [ ]:
header_numeric

### State Address

In [ ]:
header_strings

In [ ]:
header_strings[5] = "state_address"

In [ ]:
states_names, states_count = np.unique(loan_data_strings[:,5], return_counts = True)
states_count_sorted = np.argsort(-states_count)
states_names[states_count_sorted], states_count[states_count_sorted]

In [ ]:
loan_data_strings[:,5] = np.where(loan_data_strings[:,5] == '', 
                                  0, 
                                  loan_data_strings[:,5])

In [ ]:
states_west = np.array(['WA', 'OR','CA','NV','ID','MT', 'WY','UT','CO', 'AZ','NM','HI','AK'])
states_south = np.array(['TX','OK','AR','LA','MS','AL','TN','KY','FL','GA','SC','NC','VA','WV','MD','DE','DC'])
states_midwest = np.array(['ND','SD','NE','KS','MN','IA','MO','WI','IL','IN','MI','OH'])
states_east = np.array(['PA','NY','NJ','CT','MA','VT','NH','ME','RI'])

https://www2.census.gov/geo/pdfs/maps-data/maps/reference/us_regdiv.pdf

In [ ]:
loan_data_strings[:,5] = np.where(np.isin(loan_data_strings[:,5], states_west), 1, loan_data_strings[:,5])
loan_data_strings[:,5] = np.where(np.isin(loan_data_strings[:,5], states_south), 2, loan_data_strings[:,5])
loan_data_strings[:,5] = np.where(np.isin(loan_data_strings[:,5], states_midwest), 3, loan_data_strings[:,5])
loan_data_strings[:,5] = np.where(np.isin(loan_data_strings[:,5], states_east), 4, loan_data_strings[:,5])

In [ ]:
np.unique(loan_data_strings[:,5])

## Converting to Numbers

In [ ]:
loan_data_strings

In [ ]:
loan_data_strings = loan_data_strings.astype(np.int)

In [ ]:
loan_data_strings

### Checkpoint 1: Strings

In [ ]:
checkpoint_strings = checkpoint("Checkpoint-Strings", header_strings, loan_data_strings)

In [ ]:
checkpoint_strings["header"]

In [ ]:
checkpoint_strings["data"]

In [ ]:
np.array_equal(checkpoint_strings['data'], loan_data_strings)

## Manipulating Numeric Columns

In [ ]:
loan_data_numeric

In [ ]:
np.isnan(loan_data_numeric).sum()

### Substitute "Filler" Values

In [ ]:
header_numeric

#### ID

In [ ]:
temporary_fill

In [ ]:
np.isin(loan_data_numeric[:,0], temporary_fill)

In [ ]:
np.isin(loan_data_numeric[:,0], temporary_fill).sum()

In [ ]:
header_numeric

#### Temporary Stats

In [ ]:
temporary_stats[:, columns_numeric]

#### Funded Amount

In [ ]:
loan_data_numeric[:,2]

In [ ]:
loan_data_numeric[:,2] = np.where(loan_data_numeric[:,2] == temporary_fill, 
                                  temporary_stats[0, columns_numeric[2]],
                                  loan_data_numeric[:,2])
loan_data_numeric[:,2]

In [ ]:
temporary_stats[0,columns_numeric[3]]

#### Loaned Amount, Interest Rate, Total Payment, Installment

In [ ]:
header_numeric

In [ ]:
for i in [1,3,4,5]:
    loan_data_numeric[:,i] = np.where(loan_data_numeric[:,i] == temporary_fill,
                                      temporary_stats[2, columns_numeric[i]],
                                      loan_data_numeric[:,i])

In [ ]:
loan_data_numeric

### Currency Change

#### The Exchange Rate

In [ ]:
EUR_USD = np.genfromtxt("EUR-USD.csv", delimiter = ',', autostrip = True, skip_header = 1, usecols = 3)
EUR_USD

In [ ]:
loan_data_strings[:,0]

In [ ]:
exchange_rate = loan_data_strings[:,0]

for i in range(1,13):
    exchange_rate = np.where(exchange_rate == i,
                             EUR_USD[i-1],
                             exchange_rate)    

exchange_rate = np.where(exchange_rate == 0,
                         np.mean(EUR_USD),
                         exchange_rate)

exchange_rate

In [ ]:
exchange_rate.shape

In [ ]:
loan_data_numeric.shape

In [ ]:
exchange_rate = np.reshape(exchange_rate, (10000,1))

In [ ]:
loan_data_numeric = np.hstack((loan_data_numeric, exchange_rate))

In [ ]:
header_numeric = np.concatenate((header_numeric, np.array(['exchange_rate'])))
header_numeric

#### From USD to EUR

In [ ]:
header_numeric

In [ ]:
columns_dollar = np.array([1,2,4,5])

In [ ]:
loan_data_numeric[:,6]

In [ ]:
for i in columns_dollar:
    loan_data_numeric = np.hstack((loan_data_numeric, np.reshape(loan_data_numeric[:,i] / loan_data_numeric[:,6], (10000,1))))

In [ ]:
loan_data_numeric.shape

In [ ]:
loan_data_numeric

#### Expanding the header

In [ ]:
header_additional = np.array([column_name + '_EUR' for column_name in header_numeric[columns_dollar]])

In [ ]:
header_additional

In [ ]:
header_numeric = np.concatenate((header_numeric, header_additional))

In [ ]:
header_numeric

In [ ]:
header_numeric[columns_dollar] = np.array([column_name + '_USD' for column_name in header_numeric[columns_dollar]])

In [ ]:
header_numeric

In [ ]:
columns_index_order = [0,1,7,2,8,3,4,9,5,10,6]

In [ ]:
header_numeric = header_numeric[columns_index_order]

In [ ]:
loan_data_numeric

In [ ]:
loan_data_numeric = loan_data_numeric[:,columns_index_order]

### Interest Rate

In [ ]:
header_numeric

In [ ]:
loan_data_numeric[:,5]

In [ ]:
loan_data_numeric[:,5] = loan_data_numeric[:,5]/100

In [ ]:
loan_data_numeric[:,5]

### Checkpoint 2: Numeric

In [ ]:
checkpoint_numeric = checkpoint("Checkpoint-Numeric", header_numeric, loan_data_numeric)

In [ ]:
checkpoint_numeric['header'], checkpoint_numeric['data']

## Creating the "Complete" Dataset

In [ ]:
checkpoint_strings['data'].shape

In [ ]:
checkpoint_numeric['data'].shape

In [ ]:
loan_data = np.hstack((checkpoint_numeric['data'], checkpoint_strings['data']))

In [ ]:
loan_data

In [ ]:
np.isnan(loan_data).sum()

In [ ]:
header_full = np.concatenate((checkpoint_numeric['header'], checkpoint_strings['header']))

## Sorting the New Dataset

In [ ]:
loan_data = loan_data[np.argsort(loan_data[:,0])]

In [ ]:
loan_data

In [ ]:
np.argsort(loan_data[:,0])

## Storing the New Dataset

In [ ]:
loan_data = np.vstack((header_full, loan_data))

In [ ]:
np.savetxt("loan-data-preprocessed.csv", 
           loan_data, 
           fmt = '%s',
           delimiter = ',')